## Split drone images in subimages of size $(256,256)$ and store subimages as .npy 

Check if folders are in bucket

In [1]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = '003d6b042de536a0000000004'
application_key = 'K003E5Cr+BAYlvSHfg2ynLtvS5aNq78'
b2_api.authorize_account("production", application_key_id, application_key)

bucket_name = 'perturbed-minds' 
bucket = b2_api.get_bucket_by_name(bucket_name)

# List files
for file_info, folder_name in bucket.ls(show_versions=False):
    print(file_info.file_name, file_info.upload_timestamp, folder_name)

./images/test_image_up.NEF 1610469143000 ./
AIpipeline.ipynb 1610447040000 None
DJI_0001.DNG 1610975564000 None
DJI_0001.png 1610536446000 None
DJI_0001_vehicles.png 1610536314000 None
DJI_0002.DNG 1610975564000 None
DJI_0003.DNG 1610975564000 None
DJI_0003.png 1610536451000 None
DJI_0003_vehicles.png 1610536315000 None
DJI_0004.DNG 1610975565000 None
DJI_0004.png 1610536453000 None
DJI_0004_vehicles.png 1610536316000 None
DJI_0005.DNG 1610975565000 None
DJI_0005.png 1610536455000 None
DJI_0005_vehicles.png 1610536318000 None
DJI_0006.DNG 1610975568000 None
DJI_0007.DNG 1610975569000 None
DJI_0007.png 1610536456000 None
DJI_0007_vehicles.png 1610536319000 None
DJI_0008.DNG 1610975572000 None
DJI_0009.DNG 1610975576000 None
DJI_0009.png 1610536459000 None
DJI_0009_vehicles.png 1610536319000 None
DJI_0010.DNG 1610975578000 None
DJI_0010.png 1610536463000 None
DJI_0010_vehicles.png 1610536321000 None
DJI_0011.DNG 1610975579000 None
DJI_0011.png 1610536466000 None
DJI_0011_vehicles.png 161

Download images and store the raw images in './data/DroneImages/input_raw/ and the target in './data/DroneImages/target/' :

In [ ]:
# Download file
def b2_download_file(bucket, download_dest, file_name):
    b2_file_name = file_name
    local_file_name = download_dest + file_name
    download_dest = DownloadDestLocalFile(local_file_name)
    bucket.download_file_by_name(b2_file_name, download_dest)
    
from b2sdk.exception import FileNotPresent

for i in range(1,30):
    
    name = 'DJI_000%s.DNG'%str(i)
    dest = './data/DroneImages/input_raw/'
    
    name_vehicles = 'DJI_000%s_vehicles.png'%str(i)
    dest_vehicles = './data/DroneImages/target/'
    
    if i>9:
        name = './data/DroneImages/input_raw/DJI_00%s.DNG'%str(i)
        name_vehicles = './data/DroneImages/target/DJI_00%s_vehicles.png'%str(i)
    try: 
        b2_download_file(bucket, dest, name)
        b2_download_file(bucket, dest_vehicles, name_vehicles)
    except FileNotPresent:
        continue  

Store the sub-images of the the splitted input and the splitted target as .npy files in './data/DroneImages/sub_input_numpy/' and './data/DroneImages/sub_target_numpy/'. The name of the .npy file with its label is written in two of the three .csv files './data/DroneImages/car.csv', './data/DroneImages/no_car_random_sate_%s.csv' and './data/DroneImages/all.csv' according to the label.

In [17]:
import numpy as np
import pandas as pd
import rawpy


import os
from os import listdir

from pylab import *

import colour
from colour.plotting import *

from utils.dataset_generator import *

## Visual and plotting libraries
import matplotlib.pyplot as plt

In [18]:
ROI = 256

input_path = "./data/DroneImages/input_raw/"
target_path = "./data/DroneImages/target/"

input_paths = np.sort(np.array([input_path+file for file in os.listdir(input_path)]))
target_paths = np.sort(np.array([target_path+file for file in os.listdir(target_path)]))

sub_images_car = []
sub_images_no_car = []

for i, file_path in enumerate(input_paths):
    
    x = rawpy.imread(file_path)
    x = x.raw_image_visible[None,None,:,:]
    y = imread(target_paths[i])[None,None,:,:]
    
    x_splitted = split_img(x, ROIs = (ROI,ROI) , step= (ROI,ROI))
    y_splitted = split_img(y, ROIs = (ROI,ROI), step= (ROI,ROI))
    
    x_splitted = x_splitted.reshape(x_splitted.shape[0], x_splitted.shape[1], ROI, ROI)
    y_splitted = y_splitted.reshape(y_splitted.shape[0], y_splitted.shape[1], ROI, ROI)
    
    for i, sub_y in enumerate(y_splitted):
        
        sub_x = x_splitted[i]
        name_save_x = file_path.split('/')[-1].split('.')[0] + '_%i'%(i+1)
        np.save('./data/DroneImages/sub_input_numpy/'+name_save_x,sub_x)
        
        name_save_y = file_path.split('/')[-1].split('.')[0] + '_%i'%(i+1) + '_vehicles'
        np.save('./data/DroneImages/sub_target_numpy/'+name_save_y,sub_y)
        
        if sub_y.mean() == 0:
            sub_images_no_car.append(name_save_x)
            '''
            if i<2:
                print('no car',sub_y.mean())
                sub_input = np.squeeze(sub_x)
               # plt.imshow(sub_input)
                plot_image(sub_input/np.max(sub_input))
                plt.imshow(np.squeeze(sub_y))
               # plot_image(np.squeeze(sub_y))
                plt.show() 
            '''
        if sub_y.mean() != 0:
            sub_images_car.append(name_save_x)
            '''
            if i<2:
                print('car',sub_y.mean())
                sub_input = np.squeeze(sub_x)
              #  plt.imshow(sub_input)
                plot_image(sub_input/np.max(sub_input))
                plt.imshow(np.squeeze(sub_y))
               # plot_image(np.squeeze(sub_y))
                plt.show()
            '''
                
sub_images = sub_images_no_car + sub_images_car
df_no_car = pd.DataFrame({'file name':  sub_images_no_car , 'label': ['no car']*len(sub_images_no_car)})


df_car = pd.DataFrame({'file name':   sub_images_car, 'label': ['car']*len(sub_images_car)})
df_car.to_csv('./data/DroneImages/car.csv')

sub_images = sub_images_no_car + sub_images_car
labels = ['no car']*len(sub_images_no_car) +  ['car']*len(sub_images_car)
df_all = pd.DataFrame({'file name':  sub_images, 'label': labels})
df_all.to_csv('./data/DroneImages/all.csv')

seed = 72
df_no_car = df_no_car.sample(n=663,random_state = 72)
df_no_car.to_csv('./data/DroneImages/no_car_random_sate_%s.csv'%(seed))

Split the data randomly in trainingset and testset (or use the split in 'test663_random_state_72 11859.csv' and 'train663_random_state_72 11859.csv'):

In [ ]:
'''
def train_test_split(time_stamp, items_per_class=663,test_fraction=0.2,paths=[]):
       
        '''
        time_stamp (string): unique string to seperate different splits with same parameters
        items_per_class (int): maximum value is 663
        paths (list of strings): location of .csv file for class 'car' and .csv file for class 'no car'
        '''
        for i, class_path in enumerate(paths):
            testsize = int(items_per_class*test_fraction)
            df = pd.read_csv(class_path)
            test = df.sample(n=testsize)
            test_loc = './data/DroneImages/test%i_random_state_%s.csv'%(items_per_class,time_stamp)
            train_loc = './data/DroneImages/train%i_random_state_%s.csv'%(items_per_class,time_stamp)
            train = df.drop(test.index)
            train
            if i > 0:
                test.to_csv(test_loc, mode='a', header=False)
                train.to_csv(train_loc,mode='a', header=False)
            else:
                test.to_csv(test_loc)
                train.to_csv(train_loc)
        return train_loc, test_loc
    
time_stamp = str(seed) + ' ' + str(time.time() - int(time.time()))[2:7]

train_loc, test_loc = train_test_split(time_stamp,paths=['./data/DroneImages/car.csv','./data/DroneImages/no_car_random_sate_%s.csv'%(seed)])
train_loc, test_loc
'''